In [6]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from multiprocess import Pool
import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from conversion import images_to_volume

/home/yuncong/Brain/utilities/utilities2015.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages

/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_prep2_thumbnail/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD589/MD589_prep2_thumbnail/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD590/MD590_prep2_thumbnail/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD591/MD591_prep2_thumbnail/MD591-IHC17-2015.08.28-04.23.47_MD591_1_0049_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD592/MD592_prep2_thumbnail/MD592-N16-2015.08.22-00.50.39_MD592_2_0047_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD593/MD593_prep2_thumbnail/MD593-N15-2015.08.21-16.53.02_MD593_3_0045_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD594/MD594_prep2_thumbnail/MD594-IHC16-2015.08.26-16.11.01_MD594_1_0046_prep2_thumbnail.tif
/media/

Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.


In [7]:
detector_id = 998

In [8]:
out_resolution_um = 10.
resolution = '%.1fum' % out_resolution_um

In [5]:
# def load_downscaled_scoremaps_multiple_sections_sequential(sections, stack, structure, downscale, classifier_id):
#     scoremaps = {}
#     for sec in sections:
#         try:
#             actual_setting = resolve_actual_setting(setting=classifier_id, stack=stack, sec=sec)
#             sm = DataManager.load_downscaled_scoremap(stack=stack, section=sec, structure=structure, downscale=downscale, setting=actual_setting)
#             if sm is not None:
#                 scoremaps[sec] = sm
#         except:
#             pass
#     return scoremaps

# Regular

In [9]:
from skimage.filters import gaussian

In [10]:
smooth_gradient = True # Whether gradient is computed based on smoothed score volume.

In [11]:
# for stack in ['ChatCryoJane201710']:
for stack in ['MD661']:
    
    stack_spec = dict(prep_id=None,
                detector_id=detector_id,
                vol_type='score',
                structure=None,
                name=stack,
                resolution=resolution)
    
#     first_sec, last_sec = metadata_cache['section_limits'][stack]
    
#     for structure in all_known_structures:
    for structure in ['7N']:
        print structure
    
        t = time.time()
#         scoremaps = load_scoremaps_multiple_sections_parallel(stack=stack, sections=range(first_sec, last_sec+1), 
#                                                         structure=structure, downscale=downscale, classifier_setting=classifier_id)
#         scoremaps = load_downscaled_scoremaps_multiple_sections_sequential(stack=stack, sections=range(first_sec, last_sec+1), 
#                                                 structure=structure, downscale=downscale, classifier_setting=classifier_id)

        scoremaps = {}
        for sec in metadata_cache['valid_sections'][stack]:
            scoremaps[sec] = DataManager.load_downscaled_scoremap(stack=stack, section=sec, structure=structure, 
                                                      out_resolution_um=out_resolution_um, detector_id=detector_id)
        
        if len(scoremaps) < 2:
            sys.stderr.write('Number of valid scoremaps for %s is less than 2.\n' % structure)
            continue
        
        sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t)) # 10-40s (down=32, 12 processes)
        
        t = time.time()
        score_volume, score_volume_bbox_wrt_brainstem = images_to_volume(images=scoremaps, 
                            voxel_size=(1, 1, SECTION_THICKNESS/out_resolution_um))
#                                                            first_sec=first_sec-1, last_sec=last_sec-1)
        sys.stderr.write('Create score volume: %.2f seconds\n' % (time.time() - t)) # 2s
        
        print 'score_volume_bbox_wrt_brainstem =', score_volume_bbox_wrt_brainstem
        
        t = time.time()

        score_volume_filepath = DataManager.get_score_volume_filepath_v3(stack_spec, structure=structure)
        create_parent_dir_if_not_exists(score_volume_filepath)
        bp.pack_ndarray_file(score_volume.astype(np.float16), score_volume_filepath)
        upload_to_s3(score_volume_filepath)
        
        score_volume_bbox_filepath = DataManager.get_score_volume_bbox_filepath_v3(stack_spec, wrt='brainstem', structure=structure)
        np.savetxt(score_volume_bbox_filepath, np.array(score_volume_bbox_wrt_brainstem)[None], fmt='%d')
        upload_to_s3(score_volume_bbox_filepath)
        
        del scoremaps
        
        sys.stderr.write('Save score volume: %.2f seconds\n' % (time.time() - t)) # 1s (down=32)
        
        
        t = time.time()
        
        if smooth_gradient:
            gy, gx, gz = np.gradient(gaussian(score_volume, 3).astype(np.float32), 3, 3, 3)
        else:
            gy, gx, gz = np.gradient(score_volume.astype(np.float32), 3, 3, 3)
        # 3.3 second - re-computing is much faster than loading
        # .astype(np.float32) is important;
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 2s (float32) vs. 20s (float16)
        sys.stderr.write('Compute gradients %s: %f seconds\n' % (structure, time.time() - t))
        del score_volume

        t = time.time()

        gx_fp = DataManager.get_volume_gradient_filepath_v3(stack_spec, structure=structure, suffix='gx')
        gy_fp = DataManager.get_volume_gradient_filepath_v3(stack_spec, structure=structure, suffix='gy')
        gz_fp = DataManager.get_volume_gradient_filepath_v3(stack_spec, structure=structure, suffix='gz')
        
        create_parent_dir_if_not_exists(gx_fp)
        bp.pack_ndarray_file(gx, gx_fp)
        bp.pack_ndarray_file(gy, gy_fp)
        bp.pack_ndarray_file(gz, gz_fp)
        del gx, gy, gz

        upload_to_s3(gx_fp)
        upload_to_s3(gy_fp)
        upload_to_s3(gz_fp)

        sys.stderr.write('Save gradients %s: %f seconds\n' % (structure, time.time() - t))

7N
rm -rf "/home/yuncong/CSHL_scoremaps/10.0um/MD661/MD661_prep2_10.0um_detector998/MD662&661-F109-2017.06.07-01.37.37_MD661_1_0325_prep2_10.0um_detector998/MD662&661-F109-2017.06.07-01.37.37_MD661_1_0325_prep2_10.0um_detector998_7N_scoremap.bp" && mkdir -p "/home/yuncong/CSHL_scoremaps/10.0um/MD661/MD661_prep2_10.0um_detector998/MD662&661-F109-2017.06.07-01.37.37_MD661_1_0325_prep2_10.0um_detector998"
aws s3 cp "s3://mousebrainatlas-data/CSHL_scoremaps/10.0um/MD661/MD661_prep2_10.0um_detector998/MD662&661-F109-2017.06.07-01.37.37_MD661_1_0325_prep2_10.0um_detector998/MD662&661-F109-2017.06.07-01.37.37_MD661_1_0325_prep2_10.0um_detector998_7N_scoremap.bp" "/home/yuncong/CSHL_scoremaps/10.0um/MD661/MD661_prep2_10.0um_detector998/MD662&661-F109-2017.06.07-01.37.37_MD661_1_0325_prep2_10.0um_detector998/MD662&661-F109-2017.06.07-01.37.37_MD661_1_0325_prep2_10.0um_detector998_7N_scoremap.bp"


Exception: No scoremap for image MD662&661-F109-2017.06.07-01.37.37_MD661_1_0325 (section 280) for label 7N


In [ ]:
# Temp: scale atlas volumes to 10.0um

In [27]:
stack_m_spec = dict(name='atlasV5',
                   vol_type='score',
                   detector_id=None,
                   prep_id=None,
                   structure=None,
                   resolution='down32')

In [28]:
stack_m_spec_10um = dict(name='atlasV5',
                   vol_type='score',
                   detector_id=None,
                   prep_id=None,
                   structure=None,
                   resolution='10.0um')

In [35]:
for name_u in all_known_structures_sided_with_surround:
    print name_u
    
    t = time.time()
    
    v, b = DataManager.load_original_volume_v2(stack_spec=stack_m_spec, structure=name_u, bbox_wrt='atlasSpace')
    
    v_10um = rescale_by_resampling(v, XY_PIXEL_DISTANCE_TB / 10.)
    v_10um_fp = DataManager.get_original_volume_filepath_v2(stack_m_spec_10um, structure=name_u)
    create_parent_dir_if_not_exists(v_10um_fp)
    bp.pack_ndarray_file(v_10um, v_10um_fp)
    upload_to_s3(v_10um_fp)
    
    b_10um = np.round(b * XY_PIXEL_DISTANCE_TB / 10.).astype(np.int)
    b_10um_fp = DataManager.get_original_volume_bbox_filepath_v2(stack_m_spec_10um, structure=name_u, wrt='atlasSpace')
    np.savetxt(b_10um_fp, b_10um, fmt='%d')
    upload_to_s3(b_10um_fp)
    
    sys.stderr.write('Rescale %s.\n' % name_u)

10N_L
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_L.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_L.bp"
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_L_bbox_wrt_atlasSpace.txt" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_L_bbox_wrt_atlasSpace.txt"


Rescale 10N_L.


10N_L_surround_200
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_L_surround_200.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_L_surround_200.bp"
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_L_surround_200_bbox_wrt_atlasSpace.txt" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_L_surround_200_bbox_wrt_atlasSpace.txt"


Rescale 10N_L_surround_200.


10N_R
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_R.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_R.bp"
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_R_bbox_wrt_atlasSpace.txt" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_10N_R_bbox_wrt_atlasSpace.txt"


...

Rescale VLL_L_surround_200.


VLL_R
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_VLL_R.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_VLL_R.bp"
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_VLL_R_bbox_wrt_atlasSpace.txt" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_VLL_R_bbox_wrt_atlasSpace.txt"


Rescale VLL_R.


VLL_R_surround_200
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_VLL_R_surround_200.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_VLL_R_surround_200.bp"
aws s3 cp "/home/yuncong/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_VLL_R_surround_200_bbox_wrt_atlasSpace.txt" "s3://mousebrainatlas-data/CSHL_volumes/atlasV5/atlasV5_10.0um_scoreVolume/score_volumes/atlasV5_10.0um_scoreVolume_VLL_R_surround_200_bbox_wrt_atlasSpace.txt"


Rescale VLL_R_surround_200.


# Alternating

In [5]:
for stack in ['MD657']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]

    nissl_sections = []
    for sec in range(first_sec, last_sec):
        fn = metadata_cache['sections_to_filenames'][stack][sec]
        if not is_invalid(fn) and fn.split('-')[1][0] == 'N':
            nissl_sections.append(sec)
    
    for structure in all_known_structures:
#     for structure in ['7N']:
        
        print structure
    
        t = time.time()
        scoremaps = load_downscaled_scoremaps_multiple_sections_sequential(stack=stack, sections=nissl_sections, 
                                                              structure=structure, downscale=downscale, 
                                                              classifier_id=classifier_id)
        print len(scoremaps), 'scoremaps loaded.'
        if len(scoremaps) < 2:
            sys.stderr.write('Number of valid scoremaps for %s is less than 2.\n' % structure)
            continue
        
        sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t)) # 10-40s (down=32, 12 processes)
        
        t = time.time()
        score_volume, score_volume_bbox = images_to_volume(images=scoremaps, voxel_size=(1, 1, voxel_z_size), 
                                                           first_sec=np.min(nissl_sections)-1, 
                                                           last_sec=np.max(nissl_sections)-1)
        sys.stderr.write('Create score volume: %.2f seconds\n' % (time.time() - t)) # 6s (sequential)
        
        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale, 
                                                                      structure=structure, 
                                                                      classifier_setting=classifier_id)
        create_parent_dir_if_not_exists(score_volume_filepath)
        bp.pack_ndarray_file(score_volume.astype(np.float16), score_volume_filepath)

        score_volume_bbox_filepath = DataManager.get_score_volume_bbox_filepath(stack=stack, downscale=downscale, structure=structure,
                                                                               classifier_setting=classifier_id)
        np.savetxt(score_volume_bbox_filepath, np.array(score_volume_bbox)[None], fmt='%d')
        
        upload_from_ec2_to_s3(score_volume_filepath)
        upload_from_ec2_to_s3(score_volume_bbox_filepath)
        
        del score_volume, scoremaps

5N
rm -rf /shared/CSHL_scoremaps_down32/MD657/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32_5N_denseScoreMap_setting_30.bp && mkdir -p /shared/CSHL_scoremaps_down32/MD657/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32
aws s3 cp s3://mousebrainatlas-data/CSHL_scoremaps_down32/MD657/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32_5N_denseScoreMap_setting_30.bp /shared/CSHL_scoremaps_down32/MD657/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N66-2017.03.21-14.16.35_MD657_2_0197_alignedTo_MD657-F44-2017.02.18-06.0

Child returned 0
Child returned 1
0.35 seconds.
Child returned 0


rm -rf /shared/CSHL_scoremaps_down32/MD657/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32_5N_denseScoreMap_setting_30.bp && mkdir -p /shared/CSHL_scoremaps_down32/MD657/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32
aws s3 cp s3://mousebrainatlas-data/CSHL_scoremaps_down32/MD657/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32_5N_denseScoreMap_setting_30.bp /shared/CSHL_scoremaps_down32/MD657/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped_down32/MD657-N71-2017.02.22-18.34.41_MD657_2_0212_alignedTo_MD657-F44-2017.02.18-06.06.2

Child returned 1
0.35 seconds.
Load scoremaps: 0.87 seconds


127 scoremaps loaded.


...

Create score volume: 2.28 seconds


aws s3 cp /shared/CSHL_volumes/MD657/MD657_down32_scoreVolume_clf_30/score_volumes/MD657_down32_scoreVolume_clf_30_IC.bp s3://mousebrainatlas-data/CSHL_volumes/MD657/MD657_down32_scoreVolume_clf_30/score_volumes/MD657_down32_scoreVolume_clf_30_IC.bp


Child returned 0
1.46 seconds.


aws s3 cp /shared/CSHL_volumes/MD657/MD657_down32_scoreVolume_clf_30/score_volumes/MD657_down32_scoreVolume_clf_30_IC_bbox.txt s3://mousebrainatlas-data/CSHL_volumes/MD657/MD657_down32_scoreVolume_clf_30/score_volumes/MD657_down32_scoreVolume_clf_30_IC_bbox.txt


Child returned 0
0.36 seconds.


# Read

In [25]:
for structure in all_known_structures:
    print structure
    try:
        score_volume = DataManager.load_score_volume(stack='MD642', structure=structure, downscale=32, 
                                                 classifier_setting=classifier_setting)
        score_volume_cropped = score_volume[:, :600, :]

        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale, 
                                                                  structure=structure, 
                                                                  classifier_setting=classifier_setting)
        create_if_not_exists(os.path.dirname(score_volume_filepath))
        bp.pack_ndarray_file(score_volume_cropped.astype(np.float16), score_volume_filepath)
    except:
        pass

5N
6N
7N
7n
Amb
LC
LRt
Pn
Tz
VLL
RMC
SNC
SNR
3N
4N
Sp5I
Sp5O
Sp5C
PBG
10N
VCA
VCP
DC
AP
12N
RtTg
sp5
outerContour
SC


File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD642/MD642_down32_scoreVolume_clf_2/score_volumes/MD642_down32_scoreVolume_clf_2_sp5.bp
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD642/MD642_down32_scoreVolume_clf_2/score_volumes/MD642_down32_scoreVolume_clf_2_outerContour.bp


IC


In [ ]:
display_volume_sections(score_volume_cropped, cmap=plt.cm.hot, every=10)